In [1]:
#if (!requireNamespace("BiocManager", quietly = TRUE))
#  install.packages("BiocManager")
#BiocManager::install("iterativeBMA")
#install.packages("PDtoolkit")
# install.packages("remotes")
# remotes::install_github("ayhandis/creditR")

##########
# There doesn't appear to be an R implementation of the Spiegelhalter test.
# The SAS implementation seems to be very limited in its scope: it doesn't let you
# specify the hypothesized success probabilities as far as I can tell.

In [2]:
library(PDtoolkit, quietly=T)

options(warn=-1)

Warning message:
"replacing previous import 'lifecycle::last_warnings' by 'rlang::last_warnings' when loading 'tibble'"

Attaching package: 'dplyr'


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union



Attaching package: 'Hmisc'


The following objects are masked from 'package:dplyr':

    src, summarize


The following objects are masked from 'package:base':

    format.pval, units



Attaching package: 'PDtoolkit'


The following object is masked from 'package:stats':

    power




In [3]:
# Import data
test_data_main <- read.csv("pd_test_data_main.csv")
test_data_period_2 <- read.csv("pd_test_data_period_2.csv")

In [8]:
# Aggregate the results
bin_data <- test_data_main %>%
            group_by(ratings) %>%
            summarise_at(vars(predicted_pd, default_flag), funs(mean(.), sum(.), n()))

bin_data

ratings,predicted_pd_mean,default_flag_mean,predicted_pd_sum,default_flag_sum,predicted_pd_n,default_flag_n
<chr>,<dbl>,<dbl>,<dbl>,<int>,<int>,<int>
A,0.10342906,0.13546798,41.9922,55,406,406
B,0.09952396,0.08541667,47.7715,41,480,480
C,0.09571932,0.08771930,10.9120,10,114,114


### 1. Binomial / Jeffrey's / Hoshmer-Lemeshow / z-score

In [4]:


# Test with PDtoolkit
pp.testing(rating.label = bin_data$ratings,
           pdc = bin_data$predicted_pd_mean,
           no = bin_data$predicted_pd_n,
           nb = bin_data$default_flag_sum, 
           alpha = 0.05)

rating,no,nb,odr,pdc,alpha,binomial,binomial.res,jeffreys,jeffreys.res,zscore,zscore.res,hosmer.lemeshow,hosmer.lemeshow.res
<chr>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<chr>
A,406,55,0.13546798,0.10342906,0.05,0.02389227,H1: ODR > PDC,0.01995857,H1: ODR > PDC,0.01700479,H1: ODR > PDC,0.13025,H0: ODR <= PDC
B,480,41,0.08541667,0.09952396,0.05,0.86744061,H0: ODR <= PDC,0.84955196,H0: ODR <= PDC,0.84906675,H0: ODR <= PDC,0.13025,H0: ODR <= PDC
C,114,10,0.08771930,0.09571932,0.05,0.66055279,H0: ODR <= PDC,0.59864873,H0: ODR <= PDC,0.61421823,H0: ODR <= PDC,0.13025,H0: ODR <= PDC


### 2. Brier Score

In [12]:
library(iterativeBMA, quietly = T)

brier.score(bin_data$predicted_pd_mean, bin_data$default_flag_mean)

[1] 0.001289508

### 3. Herfindahl

In [9]:
library(creditR, quietly = T)

h <- Herfindahl.Hirschman.Index(bin_data, "default_flag_n")
h

# A tibble: 3 x 10
  ratings predicted_pd_mean default_flag_mean predicted_pd_sum default_flag_sum
  <chr>               <dbl>             <dbl>            <dbl>            <int>
1 A                  0.103             0.135              42.0               55
2 B                  0.0995            0.0854             47.8               41
3 C                  0.0957            0.0877             10.9               10
# ... with 5 more variables: predicted_pd_n <int>, default_flag_n <int>,
#   SumTotal <int>, concentration <df[,1]>, HHI <df[,1]>


[1] 0.408232

### 4. Spiegelhalter

In [15]:
Spiegelhalter_z = function(y, prob){
  alpha = 0.05
  z_score = sum((y-prob)*(1-2*prob))/sqrt(sum(((1-2*prob)^2)*prob*(1-prob)))
  print(z_score)
  if (abs(z_score) > qnorm(1-alpha/2)){
    print('reject null. NOT calibrated')
  } else {
    print('fail to reject. calibrated')
  }
  cat('z score: ', z_score, '\n')
  cat('p value: ', 1-pnorm(abs(z_score)), '\n')
    
  return(z_score)
}

Spiegelhalter_z(bin_data$default_flag_mean, bin_data$predicted_pd_mean)


[1] 0.01840817
[1] "fail to reject. calibrated"
z score:  0.01840817 
p value:  0.4926566 


[1] 0.01840817

In [10]:
brier.score

function (predictedArr, truthArr) 
{
    if (length(predictedArr) != length(truthArr)) {
        print("ERROR: length NOT equal!!")
    }
    temp.vec <- (truthArr - predictedArr)^2
    sum(temp.vec)
}
<bytecode: 0x0000000032c16b18>
<environment: namespace:iterativeBMA>

In [13]:
pp.testing

function (rating.label, pdc, no, nb, alpha = 0.05) 
{
    l <- list(rating.label, pdc, no, nb)
    l.check <- !length(table(sapply(l, length))) == 1
    if (l.check) {
        stop("arguments rating.label, pdc, no and nb have to be of the same length.")
    }
    if (any(!(is.numeric(pdc) | is.numeric(n) | is.numeric(nb)))) {
        stop("All arguments have to of numeric type.")
    }
    if (any(pdc > 1 | pdc < 0)) {
        stop("pdc and odr have to be between 0 and 1.")
    }
    if (any(nb > no)) {
        stop("Number of defaults cannot be greater than number of observations.")
    }
    if (any(nb < 0 | no < 0)) {
        stop("no and nb arguments cannot be negative.")
    }
    if (alpha > 1 | alpha < 0) {
        stop("significance level (alpha) has to be between 0 and 1.")
    }
    bino.test <- pbinom(nb - 1, no, pdc, lower.tail = FALSE)
    bino.test.res <- ifelse(bino.test >= alpha, "H0: ODR <= PDC", 
        "H1: ODR > PDC")
    jeff.test <- pbeta(pdc, nb + 0.5, no - nb + 0.5)
    jeff.test.res <- ifelse(jeff.test >= alpha, "H0: ODR <= PDC", 
        "H1: ODR > PDC")
    odr <- nb/no
    zsco.test <- zscore.test(pdc = pdc, odr = odr, no = no)
    zsco.test.res <- ifelse(zsco.test >= alpha, "H0: ODR <= PDC", 
        "H1: ODR > PDC")
    if (length(pdc) == 1) {
        hosm.test <- NA
        hosm.test.res <- "Only one rating grade."
    }
    else {
        hosm.test <- hl.test(pdc = pdc, no = no, nb = nb)
        hosm.test.res <- ifelse(hosm.test >= alpha, "H0: ODR <= PDC", 
            "H1: ODR > PDC")
    }
    res <- data.frame(rating = rating.label, no = no, nb = nb, 
        odr = odr, pdc = pdc, alpha = alpha, binomial = bino.test, 
        binomial.res = bino.test.res, jeffreys = jeff.test, jeffreys.res = jeff.test.res, 
        zscore = zsco.test, zscore.res = zsco.test.res, hosmer.lemeshow = hosm.test, 
        hosmer.lemeshow.res = hosm.test.res)
    return(res)
}
<bytecode: 0x000000002b057f58>
<environment: namespace:PDtoolkit>

In [ ]:
hl.test <- function(pdc, no, nb) {
	k <- length(no)
	hl <- sum((nb - no * pdc)^2 / (no * pdc * (1 - pdc)), na.rm = T)
	p.val <- pchisq(q = hl, df = k, lower.tail = FALSE)
return(p.val)
}